In [ ]:
"""Convert Ben Sira XML to HTML for proof-reading.
"""
from pathlib import Path
from lxml import etree
from typing import TextIO

def iterate_element(element: etree._Element, html_file: TextIO) -> None:
    in_reconstructed = False
    reconstructed_buffer = ""
    for child in element:
        if child.tag == "line":
            html_file.write(f"<sup id=\"line\">&#x200F;{child.attrib['n']}</sup>")
        if child.tag == "stich":
            html_file.write(f"<span style=\"display:inline-block; width: 30px;\"></span>")
        if child.tag == "w":
            rec = child.attrib.get("reconstructed", "0")
            # Si c'est un mot reconstruit
            if rec == "1":
                if not in_reconstructed:
                    in_reconstructed = True
                    reconstructed_buffer += "["
                # Ajoute le texte du mot reconstruit et de ses <g>
                if child.text:
                    reconstructed_buffer += child.text
                for g in child.findall("g"):
                    if g.text:
                        reconstructed_buffer += g.text
                reconstructed_buffer += " "
            else:
                # Si on sort d'une séquence reconstruite, ferme le crochet
                if in_reconstructed:
                    reconstructed_buffer = reconstructed_buffer.rstrip() + "] "
                    html_file.write(reconstructed_buffer)
                    reconstructed_buffer = ""
                    in_reconstructed = False
                if child.text:
                    html_file.write(f"{child.text} ")
                for grandchild in child:
                    if grandchild.tag == "del":
                        if grandchild.text:
                            html_file.write(f"<del>{grandchild.text}</del> ")
                    if grandchild.tag == "hi":
                        if grandchild.text:
                            html_file.write(f"<span class=\"highlight\" id=\"{grandchild.attrib.get("rend")}\">{grandchild.text}</span> ")
                    if grandchild.tag == "space":
                        html_file.write("_" * int(grandchild.attrib.get("extent", "1")))
                    if grandchild.tag == "stich":
                        html_file.write(f"<span style=\"display:inline-block; width: 30px;\"></span>")                
    # Si on arrive à la fin, ferme le crochet si besoin
    if in_reconstructed:
        reconstructed_buffer = reconstructed_buffer.rstrip() + "] "
        html_file.write(reconstructed_buffer)
                    


def iterate_tree(parsed_manuscript: etree._Element, html_file: TextIO) -> None:
    for element in parsed_manuscript.iter():
        if element.tag == "ms":
            html_file.write(
                f"""<!DOCTYPE html>\n<html>\n<head>\n<link rel="stylesheet" href="styles.css">\n<title>{element.attrib['name']}</title>\n</head>\n<body><h1>{element.attrib['name']}</h1>\n"""
            )
        if element.tag == "div":
            if element.attrib.get("type") == "chap":
                html_file.write(f"<h2>Chapter {element.attrib['n']}</h2>")
            if element.attrib.get("type") == "verse":
                html_file.write(f"<p dir=\"rtl\"><b id=\"verse\">{element.attrib['n']}</b>\n")
                iterate_element(element, html_file)
                html_file.write(f"</p>\n")
        if element.tag == "margin":
             html_file.write(f"""
            <p dir="rtl" class="margin" id="{element.attrib.get("type")}">
            <sup id="line">&#x200F;{element.attrib.get("line")}</sup>
            """)
             iterate_element(element, html_file)
             html_file.write(f"</p>\n")
             



if __name__ == "__main__":
    #Lecture de l'ensemble des fichier xml
    for file_path in Path("../xml_tei_revised").glob("*.xml"):
        print(f"Converting {file_path} to HTML...")
        parsed_manuscript = etree.parse(str(file_path)).getroot()
        
        # Create a new HTML file for each
        with open(f"htmls_lxml/{Path(file_path).stem}.html", "w", encoding="utf-8") as html_file:
            # Iterate over the XML elements
            iterate_tree(parsed_manuscript, html_file)

Converting ../xml_tei_revised/ms_b.xml to HTML...
Converting ../xml_tei_revised/ms_c.xml to HTML...
Converting ../xml_tei_revised/ms_a.xml to HTML...
Converting ../xml_tei_revised/ms_d.xml to HTML...
Converting ../xml_tei_revised/ms_e.xml to HTML...
Converting ../xml_tei_revised/ms_f.xml to HTML...
Converting ../xml_tei_revised/ms_11QPsa.xml to HTML...
Converting ../xml_tei_revised/ms_massada.xml to HTML...
Converting ../xml_tei_revised/ms_2Q18.xml to HTML...
